# Imports

In [34]:
import torch
import numpy as np

from torch import Tensor
from scipy.stats import kendalltau

# Define Variables

In [48]:
X = [[ 0,  0,  0,  0,  0],
     [ 1,  1,  1,  1,  1],
     [ 2,  2,  2,  2,  2],
     [ 3,  3,  3,  3,  3],
     [ 4,  4,  4,  4,  4],
     [ 5,  5,  5,  5,  5],
     [ 6,  6,  6,  6,  6],
     [ 7,  7,  7,  7,  7],
     [ 8,  8,  8,  8,  8],
     [ 9,  9,  9,  9,  9],
     [10, 10, 10, 10, 10],
     [11, 11, 11, 11, 11],
     [12, 12, 12, 12, 12],
     [13, 13, 13, 13, 13],
     [14, 14, 14, 14, 14],
     [15, 15, 15, 15, 15]]
Y = [[2, 4, 3, 1, 0],
     [4, 0, 1, 3, 2],
     [0, 1, 2, 4, 3],
     [2, 0, 1, 4, 3],
     [1, 3, 4, 2, 0],
     [2, 4, 1, 0, 3],
     [1, 0, 3, 4, 2],
     [0, 2, 1, 4, 3],
     [0, 1, 3, 4, 2],
     [4, 0, 2, 3, 1],
     [0, 3, 2, 1, 4],
     [1, 4, 3, 0, 2],
     [2, 1, 4, 0, 3],
     [1, 2, 0, 4, 3],
     [1, 2, 0, 4, 3],
     [3, 2, 0, 4, 1]]
scores = [[-0.0424,  0.1535,  0.0563,  0.0577,  0.1411],
          [-0.0157,  0.1476,  0.0112,  0.0521,  0.1176],
          [-0.0265,  0.1078,  0.0318,  0.0275,  0.0674],
          [-0.0081,  0.1382,  0.0486,  0.0268,  0.1309],
          [-0.0124,  0.1531,  0.0639,  0.0591,  0.1099],
          [-0.0066,  0.1747,  0.0393,  0.0804,  0.1225],
          [-0.0260,  0.1381,  0.0153,  0.0265,  0.0978],
          [ 0.0219,  0.1581,  0.0680,  0.0395,  0.1192],
          [-0.0446,  0.1284,  0.0518,  0.0464,  0.1383],
          [ 0.0035,  0.1353,  0.0764,  0.0345,  0.1323],
          [ 0.0331,  0.1531,  0.0543,  0.0525,  0.0949],
          [ 0.0140,  0.1665,  0.0922,  0.0464,  0.1693],
          [-0.0216,  0.1608,  0.0509,  0.0323,  0.0998],
          [-0.0032,  0.1082,  0.0348,  0.0208,  0.1090],
          [ 0.0100,  0.1327,  0.0487,  0.0041,  0.1105],
          [-0.0113,  0.1485,  0.0518,  0.0830,  0.1402]]
mask = [[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]]
X, Y, scores, mask = (
    torch.tensor(X), torch.tensor(Y),
    torch.tensor(scores, requires_grad=True), 
    torch.tensor(mask),
)

# Define [DiffKendall Function](https://papers.nips.cc/paper_files/paper/2023/hash/9b01333262789ea3a65a5fab4c22feae-Abstract-Conference.html)

$$
\tilde{\tau}_\alpha (\pmb{x}, \pmb{y}) =
\frac{1}{N_0} \sum\limits_{i=2}^{n}\sum\limits_{j=1}^{i-1}
\frac{e^{\alpha (x_i - x_j)} - e^{-\alpha (x_i-x_j)}}{e^{\alpha(x_i - x_j)} + e^{-\alpha(x_i - x_j)}}
\frac{e^{\alpha(y_i-y_j)} - e^{-\alpha(y_i-y_j)}}{e^{\alpha(y_i-y_j)} + e^{-\alpha(y_i-y_j)}}
$$



In [ ]:
%%timeit -r 1 -n 1
def diff_kendall(
    scores: Tensor, 
    X: Tensor, 
    Y: Tensor, 
    mask: Tensor,
    alpha: Tensor=1.,
) -> Tensor:
    rows, cols = scores.shape
    N_0 = torch.sum(mask, dim=-1)
    sums = torch.zeros(rows)
    for i in range(1, cols):
        for j in range(i):

            term1 = torch.exp(alpha*(scores[:, i] - scores[:, j]))
            term2 = torch.exp(-alpha*(scores[:, i] - scores[:, j]))
            term3 = torch.exp(alpha*(Y[:, i] - Y[:, j]))
            term4 = torch.exp(-alpha*(Y[:, i] - Y[:, j]))

            frac1 = (term1 - term2)/(term1 + term2)
            frac2 = (term3 - term4)/(term3 + term4)

            sums += (frac1*frac2)*(mask[:, i]*mask[:, j])
    
    return torch.mean((1/N_0)*sums)

tau = diff_kendall(scores, X, Y, mask)
print(f'DiffKendall: {tau}')

tensor(0.0048, grad_fn=<MeanBackward0>)
DiffKendall: 0.004750354215502739
3.94 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Check Actual $\tau$ Value

In [40]:

print(np.mean([
    kendalltau(x[mask], y[mask]).statistic 
    for x, y, mask in 
    zip(scores.detach().cpu().numpy(), Y.numpy(), mask.numpy())
]))

-0.012499999999999997
